In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
train_path = 'E:/ML Internship/cat&dog/cats_dogs_light/train'
test_path = 'E:/ML Internship/cat&dog/cats_dogs_light/test'

In [6]:
def preprocess_images(image_paths, target_size=(224, 224)):
    processed_images = []
    for path in image_paths:
        image = cv2.imread(path)
        resized_image = cv2.resize(image, target_size)
        normalized_image = resized_image / 255.0
        processed_images.append(normalized_image)
    return processed_images

def preprocess_dataset(dataset_path):
    classes = os.listdir(dataset_path)
    images = []
    labels = []
    for i, class_name in enumerate(classes):
        class_path = os.path.join(dataset_path, class_name)
        class_images = [os.path.join(class_path, img) for img in os.listdir(class_path)]
        images.extend(class_images)
        labels.extend([i] * len(class_images))  # Assign label based on class index
    return preprocess_images(images), labels

# Preprocess train, test, and val datasets
X_train, y_train = preprocess_dataset(train_path)
print("completed ")
X_test, y_test = preprocess_dataset(test_path)
print("completed ")

completed 
completed 


In [7]:
X_train = np.array(X_train)
print("completed")
y_train = np.array(y_train)
print("completed")
X_test = np.array(X_test)
print("completed")
y_test = np.array(y_test)

completed
completed
completed


In [26]:
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score
import cv2

# Assuming you have your features in X_train, X_test, y_train, and y_test

# Load the VGG16 model pre-trained on ImageNet data
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model by adding a few more layers on top of the pre-trained model
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define a checkpoint to save the model with the best validation accuracy
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, save_weights_only=False, monitor='val_accuracy', mode='max')

# Train the model for 10 epochs
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[checkpoint])

# Load the best model (model with the highest validation accuracy)
best_model = load_model('best_model.h5')

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
y_pred_binary = np.round(y_pred)  # Convert probabilities to binary predictions
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)


Epoch 1/20
3/3 [==============================] - 29s 10s/step - loss: 5.3686 - accuracy: 0.4500 - val_loss: 2.2197 - val_accuracy: 0.5000
Epoch 2/20
3/3 [==============================] - 30s 10s/step - loss: 2.1661 - accuracy: 0.5000 - val_loss: 1.9965 - val_accuracy: 0.5000
Epoch 3/20
3/3 [==============================] - 31s 11s/step - loss: 1.5544 - accuracy: 0.5000 - val_loss: 0.8420 - val_accuracy: 0.5000
Epoch 4/20
3/3 [==============================] - 33s 11s/step - loss: 0.5734 - accuracy: 0.7250 - val_loss: 0.5867 - val_accuracy: 0.7500
Epoch 5/20
3/3 [==============================] - 34s 12s/step - loss: 0.4456 - accuracy: 0.9000 - val_loss: 0.5435 - val_accuracy: 0.8000
Epoch 6/20
3/3 [==============================] - 33s 11s/step - loss: 0.3952 - accuracy: 0.9000 - val_loss: 0.4873 - val_accuracy: 0.9000
Epoch 7/20
3/3 [==============================] - 33s 11s/step - loss: 0.3047 - accuracy: 0.9875 - val_loss: 0.4600 - val_accuracy: 0.7500
Epoch 8/20
3/3 [===========

In [27]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2

# Load the saved model
loaded_model = load_model('best_model.h5')

# Function to preprocess a single image
def preprocess_image(image_path, target_size=(224, 224)):  # Adjust the target_size
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, target_size)
    normalized_image = resized_image / 255.0
    return np.expand_dims(normalized_image, axis=0)  # Add batch dimension

# Path to the new image you want to predict
new_image_path = 'C:/Users/92309/Downloads/output_faces/download.jpg'

# Preprocess the new image
new_image = preprocess_image(new_image_path)

# Make predictions using the loaded model
predictions = loaded_model.predict(new_image)
predicted_class = "cat" if predictions[0][0] < 0.5 else "dog"

print("Predicted class:", predicted_class)


1/1 [==============================] - 1s 520ms/step
Predicted class: cat
